In [2]:
# import sys
# sys.path.insert(0,'/p/project/training2005/jupyter/kernels/tensorflow_test/lib/python3.6/site-packages')
# import tensorflow

import tensorflow.keras as keras
from tensorflow.keras import layers, Input, Model
# import tensorflow.keras.layers.Attention as Attention
from functools import reduce

import matplotlib.pyplot as plt
import numpy as np
# ttd = np.load('redttd.npy')
# ptd = np.load('redptd.npy')
# ntd = np.load('/p/project/training2005/HZG_Challenge/nao_index_train.npy')

# ttst = np.load('redttst.npy')
# ptst = np.load('redptst.npy')

ttd = np.load('/p/project/training2005/HZG_Challenge/tas_train.npy')
ptd = np.load('/p/project/training2005/HZG_Challenge/psl_train.npy')
ntd = np.load('/p/project/training2005/HZG_Challenge/nao_index_train.npy')

ttst = np.load('/p/project/training2005/HZG_Challenge/tas_predict.npy')
ptst = np.load('/p/project/training2005/HZG_Challenge/psl_predict.npy')

#plt.plot(range(data.shape[1]),data[0])

def branch_graph(dim, input_tensor):
    x = layers.Dense(dim, activation='relu')(input_tensor)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.Dense(dim, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x1 = x
    x = layers.Flatten()(x)
    x1 = layers.Dense(dim, activation='sigmoid')(x1)
    x1 = layers.Flatten()(x1)
    x = layers.Multiply()([x,x1])
#     x = Attention()([x,x1])
    return x
    
def build_model():
    inp1 = Input(shape=(2322,1))
    inp2 = Input(shape=(2322,1))
    
    #first branch
    x1 = branch_graph(3,inp1)
    
    #second branch
    x2 = branch_graph(3,inp2)

    #merge branches 
    out = layers.Concatenate()([x1,x2])
#     out = layers.Flatten()(out)
    out = layers.Dense(1)(out)
    
    model = Model([inp1,inp2],out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.summary()
    return model

def train_model(model, params ={'vsplit':0.111,'ne':145,'bs':256}):
#     d1 = 
    history = model.fit([np.reshape(ttd,(ttd.shape[0],ttd.shape[1],1)),
                         np.reshape(ptd,(ptd.shape[0],ptd.shape[1],1))], 
                        np.reshape(ntd,(ntd.shape[0],1)),
                        epochs=params['ne'], 
                        batch_size=params['bs'],
                        validation_split=params['vsplit'])
    return [model, history]

def test_model(model):
    return model.predict([ttst,ptst])

def plot_losses(epochs, history):
    #ks = history.keys()
    ks =['loss','val_loss','mean_absolute_error','val_mean_absolute_error']
    [plt.plot(range(epochs), history[k],'x-') for k in ks]
    plt.show()

In [9]:
model = build_model()
keras.utils.plot_model(model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2322, 1)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2322, 1)      0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 2322, 3)      6           input_1[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 2322, 3)      6           input_2[0][0]          

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [25]:
model, loss = train_model(model)

Train on 800 samples, validate on 100 samples
Epoch 1/145
800/800 [==============================] - 5s 6ms/step - loss: 1.0211 - mean_absolute_error: 0.8094 - val_loss: 1.2592 - val_mean_absolute_error: 0.9403
Epoch 2/145
800/800 [==============================] - 0s 150us/step - loss: 0.9107 - mean_absolute_error: 0.7587 - val_loss: 1.2627 - val_mean_absolute_error: 0.9645
Epoch 3/145
800/800 [==============================] - 0s 148us/step - loss: 0.8870 - mean_absolute_error: 0.7473 - val_loss: 1.3215 - val_mean_absolute_error: 0.9463
Epoch 4/145
800/800 [==============================] - 0s 145us/step - loss: 0.8901 - mean_absolute_error: 0.7423 - val_loss: 1.3011 - val_mean_absolute_error: 0.9620
Epoch 5/145
800/800 [==============================] - 0s 146us/step - loss: 0.8604 - mean_absolute_error: 0.7435 - val_loss: 1.3274 - val_mean_absolute_error: 0.9683
Epoch 6/145
800/800 [==============================] - 0s 145us/step - loss: 0.8138 - mean_absolute_error: 0.7178 - val_l